# Testing all

In [ ]:
import scifysim as sf
import numpy as np
import matplotlib.pyplot as plt
seed = 10
asim = sf.director.simulator(fpath="local_config/default_new_4T.ini")
asim.prepare_observatory(file=asim.config)
asim.prepare_injector(file=asim.config, fpath="local_config/", seed=seed)
asim.prepare_combiner(asim.config)
asim.prepare_sequence(asim.config)
asim.prepare_fringe_tracker(asim.config, seed=seed)
asim.fringe_tracker.prepare_time_series(asim.lambda_science_range, duration=10, replace=True)

In [ ]:
sky_trans = sf.sources.transmission_emission()
VLTI_UT_trans = sf.sources.transmission_emission(trans_file="data/VLTI_UT_trans_sfs.txt")
sf.sources.chain(sky_trans, VLTI_UT_trans)
VLTI_UT_trans.downstream = None
sky_trans.upstream = None

# Manually building sources

This will probably go into a method soon

In [ ]:
from astropy import units
asim.src = sf.sources.group()
asim.obs.point(asim.sequence[10], asim.target)
print("airamss: ", asim.obs.altaz.secz)

# Building the sources
T_star = asim.config.getfloat("target", "star_temperature")
R_star = asim.config.getfloat("target", "star_radius")
T_planet = asim.config.getfloat("target", "star_temperature")
R_planet = asim.config.getfloat("target", "planet_radius") * units.Rjup.to(units.Rsun)
distance = asim.config.getfloat("target", "star_distance")
planet_separation = asim.config.getfloat("target", "planet_sep")
planet_position_angle = asim.config.getfloat("target", "planet_pa")
planet_offsetx = -planet_separation*np.sin(planet_position_angle * np.pi/180)
planet_offsety =  planet_separation*np.cos(planet_position_angle * np.pi/180)
planet_offset = (planet_offsetx, planet_offsety)
print("sep = ", planet_separation)
print("pa = ", planet_position_angle)
print("offset = ", planet_offset)

# Building the transmission chain
t_sky = asim.config.getfloat("atmo", "t_sky")
t_vlti = asim.config.getfloat("vlti", "T_vlti")

asim.src.sky = sf.sources.transmission_emission(trans_file="data/MK_trans_sfs.txt", T=t_sky, airmass=True, observatory=asim.obs)
asim.src.UT = sf.sources.transmission_emission(trans_file="data/VLTI_UT_trans_sfs.txt", T=t_vlti)
sf.sources.chain(asim.src.sky, asim.src.UT)
asim.src.UT.downstream = None
asim.src.sky.upstream = None

asim.src.star = sf.sources.resolved_source(asim.lambda_science_range,
                                           distance=distance, radius=R_star, T=T_star)
asim.src.planet = sf.sources.resolved_source(asim.lambda_science_range,
                                             distance=distance, radius=R_planet, T=T_planet,
                                             resolved=False, offset=planet_offset)




In [ ]:
asim.combiner.bright = np.array([False, False, True, False, False, True, False, False])
asim.combiner.dark = np.array([False, False, False, True, True, False, False, False])
asim.combiner.photometric = np.array([True, True, False, False, False, False, True, True])

# Scheduling / preparation

The SCIFYsim integrates with `astroplan`

In [ ]:
sf.observatory.plots.plot_altitude(asim.target, asim.obs.observatory_location, asim.sequence)
plt.title("%s altitude"%(asim.tarname))
plt.show()
sf.observatory.plots.plot_airmass(asim.target, asim.obs.observatory_location, asim.sequence)
plt.title("%s airmass"%(asim.tarname))
plt.show()
sf.observatory.plots.plot_finder_image(target=asim.target)
plt.title("%s finder view"%(asim.tarname))
plt.show()

In [ ]:
for i in range(len(asim.sequence)):
    fig = sf.plot_tools.plot_projected_pupil(asim, i,
                                             dist=100, grid=True)
    fig.show()

# Propagating some photons
## The metrologic integration
The method `metrologic_integration` of the `director` records a lot of metrology to help analyze the situation.

It also has more overhead than a typical method to compute outputs.

Here, detector noises are not computed. Only the fluctuations of the instantaneous flux are computed.

In [ ]:
t_exp = 10.
integ = asim.make_metrologic_exposure(asim.src.planet, asim.src.star, [asim.src.sky, asim.src.UT],
                                      texp=t_exp)


In [ ]:
integ.nsamples = integ.starlight.shape[0]
integ.summed_signal = integ.static[0].T[None,:,:]+\
                    integ.static[1].T[None,:,:]+integ.starlight+\
                    integ.planetlight
integ.star_sum = integ.starlight.sum(axis=0)
integ.planet_sum = integ.planetlight.sum(axis=0)
integ.sky_sum = integ.static[0].T * integ.nsamples
integ.inst_sum = integ.static[1].T * integ.nsamples

shift_step = 0.03
outputs = np.arange(integ.summed_signal.shape[2])
isources = np.arange(4)
raw_sources = [integ.static[1], integ.static[1], integ.starlight, integ.planetlight]
sources = [integ.inst_sum, integ.sky_sum, integ.star_sum, integ.planet_sum]
source_labels = ["instrument", "sky", "star", "planet"]
bottom = np.zeros_like(sources[0])
pup = 1 # The pupil for which to plot the piston
print(integ.planet_sum.shape)
fig = plt.figure()
bars = []
read_noise = integ.ron
for ksource, (thesource, label) in enumerate(zip(sources, source_labels)):
    photon_noise = np.sqrt(thesource)
    if ksource >= 2:
        inst_noise = np.std(raw_sources[ksource], axis=0)
    else:
        inst_noise = np.zeros((asim.lambda_science_range.shape[0], outputs.shape[0]))
    #print("Inst noise", ksource,  inst_noise.mean(axis=0))
    #print("Photon noise", ksource, photon_noise.mean(axis=0))
    noise = np.sqrt(photon_noise**2 + read_noise**2 + inst_noise**2)
    for ilamb in range(asim.lambda_science_range.shape[0]):
        #print(ksource, ilamb, label)
        #pdb.set_trace()
        if ilamb == 0:
            bars.append(plt.bar(outputs+shift_step*ilamb, thesource[ilamb,:], bottom=bottom[ilamb,:],
                label=label, width=shift_step, color="C%d"%ksource)) #yerr=noise[ilamb,:]
        else:
            bars.append(plt.bar(outputs+shift_step*ilamb, thesource[ilamb,:], bottom=bottom[ilamb,:],
                width=shift_step,  color="C%d"%ksource)) #yerr=noise[ilamb,:]
    bottom += thesource
#plt.legend((bars[i][0] for i in range(len(bars))), source_labels)
#Handled the legend with an condition in the loop
plt.legend()
plt.xticks(outputs)
plt.xlabel(r"Output and spectral channel %.1f to %.1f $\mu m$ ($R\approx %.0f$)"%(asim.lambda_science_range[0]*1e6,
                                                                                 asim.lambda_science_range[-1]*1e6,
                                                                                 asim.R.mean()))
plt.title("Integration of %.2f s on %s"%(t_exp, asim.tarname))
plt.ylabel("Number of photons")
plt.show()

plt.figure()
plt.violinplot(integ.summed_signal.sum(axis=1))
plt.ylabel("Number of photons per simulation step (%.0f ms)"%(asim.injector.screen[0].step_time*1000))
plt.xlabel("Outputs")
plt.title("Temporal distribution of the output illumination")
plt.show()


In [ ]:


# I should profide an easier access to this time step
integration_step = asim.injector.screen[0].step_time
t = np.arange(integ.summed_signal.shape[0])*integration_step
plt.figure()
pup = 1
plt.plot(t, integ.ft_phase[:,pup], label="Fringe tracker phase")
plt.plot(t, integ.inj_phase[:,:], label="Injection phase")
#plt.plot(asim.fringe_tracker.ref_sample_times[:1000],
#         2*np.pi/3.5e-6*asim.fringe_tracker.dry_piston_series[:1000,pup],
#        label= "Sample", alpha=0.3)
plt.title("Residual phase for pupil %d"%(pup))
plt.xlabel("Time [s]")
plt.ylabel("Phase [rad]")
plt.legend()
plt.show()

plt.figure()
plt.plot(t, integ.inj_amp[:]**2, label="Injection rate")
#plt.plot(asim.fringe_tracker.ref_sample_times[:1000],
#         2*np.pi/3.5e-6*asim.fringe_tracker.dry_piston_series[:1000,pup],
#        label= "Sample", alpha=0.3)
plt.title("Residual coupling rate")
plt.xlabel("Time [s]")
plt.ylabel("Coupling ")
plt.ylim(0,0.8)
plt.legend()
plt.show()

plt.figure()
pup = 1
plt.plot(t, integ.summed_signal.sum(axis=1)[:,3:5], label="Dark output signal")
plt.plot(t, integ.summed_signal.sum(axis=1)[:,3] - integ.summed_signal.sum(axis=1)[:,4], label="Kernel signal")
#plt.plot(asim.fringe_tracker.ref_sample_times[:1000],
#         2*np.pi/3.5e-6*asim.fringe_tracker.dry_piston_series[:1000,pup],
#        label= "Sample", alpha=0.3)
plt.title("Individual and differential outputs")
plt.xlabel("Time [s]")
plt.ylabel("Photons")
plt.legend()
plt.show()

In [ ]:
bins = np.linspace(0, 1e9, 100)
total_kernels = integ.summed_signal[:,:,3] - integ.summed_signal[:,:,4]
star_kernels = integ.starlight[:,:,3] - integ.starlight[:,:,4]
plt.figure()
plt.hist(integ.summed_signal[:,:,3].mean(axis=1), bins=bins,
         histtype="step", label="Null1")
plt.hist(integ.summed_signal[:,:,4].mean(axis=1), bins=bins,
         histtype="step", label="Null2")
plt.hist(integ.starlight[:,:,3].mean(axis=1), bins=bins,
         histtype="step", label=r"Null1 under $\mathcal{H}_0$")
plt.hist(integ.starlight[:,:,4].mean(axis=1), bins=bins,
         histtype="step", label=r"Null2 under $\mathcal{H}_0$")
plt.legend()
plt.title("Distribution of the nulls")
plt.xlabel("Photon per 5ms subexp")
plt.ylabel("Occurences")
plt.show()

In [ ]:
bins = np.linspace(-1e9, 1e9, 100)
total_kernels = integ.summed_signal[:,:,3] - integ.summed_signal[:,:,4]
star_kernels = integ.starlight[:,:,3] - integ.starlight[:,:,4]
plt.figure()
plt.hist(total_kernels.mean(axis=1), bins=bins,
         histtype="step", label="Total")
plt.hist(star_kernels.mean(axis=1), bins=bins,
         histtype="step", label=r"Under $\mathcal{H}_0$")
plt.legend()
plt.title("Distribution of the kernel-null")
plt.xlabel("Photon per 5ms subexp")
plt.ylabel("Occurences")
plt.show()